In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse, \
                            mean_absolute_error as mae, \
                            r2_score
from math import sqrt

from matplotlib import pyplot as plt
import pickle

# Метрики

Для оценки моделей используются следующие метрики: 

- Корень из среднеквадратичной ошибки (Root-Mean-Squared Error, RMSE)

    $ \mathit{RMSE} = \sqrt{ \dfrac{1}{n} \sum_{i=1}^n {(actual_i - predicted_i)^2} } $
    <br/><br/>

- Средневзвешенная абсолютная процентная ошибка (Weighted Mean Absolute Percentage Error, WMAPE)

    $ \mathit{WMAPE} = \dfrac{ \sum_{i=1}^n {\lvert actual_i - predicted_i \rvert}}{\sum_{i=1}^n {\lvert actual_i \rvert}} $
    <br/><br/>
    
- Процентное смещение (Percent Bias, PBias)

    $ \mathit{PBias} = \dfrac{ \sum_{i=1}^n {predicted_i - actual_i}}{\sum_{i=1}^n {actual_i}} $
    <br/><br/>
    
- Коэффициент детерминации (R2)

    $ \mathit{R2} = 1 - \dfrac{ \sum_{i=1}^n {(actual_i - predicted_i)^2}} {\sum_{i=1}^n {(actual_i - \overline{actual_i})^2}}$ 
    
Перечисленные метрики вычисляются для всего тестового ряда (365 дней), а также отдельно для каждого месяца (порции по 30 дней). 

Дополнительно для каждого дня вычисляется абсолютная процентная ошибка (Absolute Percent Error, APE):

$ \mathit{APE} = \lvert \dfrac{actual - predicted}{actual} \rvert $


In [8]:
def get_metrics(actual, predicted):
    return [sqrt(mse(actual, predicted)), # RMSE
               mae(actual, predicted) / np.mean(np.abs(actual)), # WMAPE
               np.mean(predicted - actual) / np.mean(actual), # PBias
               r2_score(actual, predicted)] # R2

def get_forecast_metrics(actual, predicted):    
    overall_metrics = get_metrics(actual, predicted)
    
    months_metrics = []
    
    for i in range(12):
        a = actual[30 * i : 30 * (i + 1)]
        p = predicted[30 * i : 30 * (i + 1)]
        
        months_metrics.append(get_metrics(a, p))
        
    days_metrics = []
    
    for i in range(len(actual)):
        mape = abs((predicted[i] - actual[i]) / actual[i])
        
        days_metrics.append(mape)
        
    return overall_metrics, months_metrics, days_metrics

In [9]:
def plot_forecasts(actual, forecasts, labels):
    plt.figure(figsize=(17, 10))

    plt.plot(actual, 'black', label='ground truth')
    
    for predicted, label in zip(forecasts, labels):
        plt.plot(predicted, alpha=0.8, label=label)

    plt.legend()
    plt.show()

In [10]:
def plot_months_metrics(models_months_metrics, labels):
    fig, axs = plt.subplots(2, 2, figsize=(16,8))
    fig.suptitle('Метрики по месяцам', y=0.95)
    fig.subplots_adjust(hspace=0.3)
    
    for months_metrics, label in zip(models_months_metrics, labels):
        axs[0][0].plot([m[0] for m in months_metrics], '.-', label=label)
        axs[0][1].plot([m[1] * 100 for m in months_metrics], '.-', label=label)
        axs[1][0].plot([m[2] * 100 for m in months_metrics], '.-', label=label)
        axs[1][1].plot([m[3] for m in months_metrics], '.-', label=label)
    
    axs[0][0].legend()
    axs[0][0].set_title('RMSE')

    axs[0][1].legend()
    axs[0][1].set_title('WMAPE')

    axs[1][0].legend()
    axs[1][0].set_title('PBias')

    axs[1][1].legend()
    axs[1][1].set_title('R2')

    plt.show()

In [11]:
def plot_days_metrics(models_days_metrics, labels):
    plt.figure(figsize=(14, 7))
    
    for days_metrics, label in zip(models_days_metrics, labels):
        plt.plot([i * 100 for i in days_metrics], label=label)
        
    plt.title('APE')
    plt.legend()

    plt.show()

In [12]:
def evaluate_forecasts(actual, forecasts, labels):
    models_months_metrics = []
    models_days_metrics = []
    
    print('{:^12}|{:^12s}|{:^12s}|{:^12s}|{:^12s}'.format('', 'RMSE', 'MAPE (%)', 'PBias (%)', 'R2'))
    print(('-' * 12 + '|') * 4 + '-' * 12)
    
    for forecast, label in zip(forecasts, labels):
        overall_metrics, months_metrics, days_metrics = get_forecast_metrics(actual, forecast)
        
        models_months_metrics.append(months_metrics)
        models_days_metrics.append(days_metrics)
        
        print('{:^12s}|{:^12.3f}|{:^12.3f}|{:^12.3f}|{:^12.3f}'.format(label, overall_metrics[0], overall_metrics[1] * 100,
                                                    overall_metrics[2] * 100, overall_metrics[3]))
    
    print()
    
    plot_forecasts(actual, forecasts, labels)
    plot_months_metrics(models_months_metrics, labels)
    plot_days_metrics(models_days_metrics, labels)